## Beyond numpy
Some calculation cannot be efficienclty performed with numpy
* numpy need a lot of memory
* Operation not implemented

Example : 
* Calculer $\pi$ (avec une formule très très lente!!!)
$$ \frac\pi4 = \sum_i \frac{(-1)^i}{2i+1} = 1 - \frac13 + \frac 15 - \frac17 + \ldots $$

* Operation similar to cumsum
$$ y_n = f(y_{n-1}, x_n) $$

In [1]:
from numpy import arange, sum
N = 1000000
# Implementation in pure python
def calc_pi_python(N):
    out = 0
    sgn = 1
    for i in range(N):
        out += sgn/(2*i+1)
        sgn = -sgn
    return 4*out

%timeit calc_pi_python(N)

# numpy
def calc_pi_numpy(N):
    i = arange(N)
    tmp = 1-2*(i%2)
    return 4*sum(tmp/(2*i+1))
%timeit calc_pi_numpy(N)
calc_pi_numpy(N)

10 loops, best of 3: 173 ms per loop
10 loops, best of 3: 30.8 ms per loop


3.1415916535897939

## ctypes
* Interface between python and shared library (dll, so)
* Accelerate your code (this method is not recommended)
* Use existing code !!!
* Use closed source library

No magic : you have to know C and deal with pointer, memory allocation, ...

In [4]:
import ctypes
lib = ctypes.cdll.LoadLibrary('../calc_pi/libpi.so')

# Raw function
_calc_pi = lib.calc_pi

# Wrapper to be python friendly
def calc_pi_ctypes(N):
    out = ctypes.c_double(0)
    _calc_pi(N, ctypes.byref(out))
    return out.value*4

%timeit calc_pi_ctypes(N)

100 loops, best of 3: 4.15 ms per loop


## Numba 
Compile your python code for free

In [15]:
import numba

def calc_pi_python(N):
    out = 0.
    sgn = 1
    for i in range(N):
        out =out + sgn/(2*i+1)
        sgn = -sgn
    return 4*out

calc_pi_numba = numba.jit(numba.float64(numba.int32))(calc_pi_python)


In [16]:
%timeit calc_pi_numba(N)

100 loops, best of 3: 8.46 ms per loop


In [19]:
from numpy import sin
from numpy.random import rand
a = rand(10000000)

In [20]:
%timeit b = sin(a)*a + a**2 + 1/a

1 loop, best of 3: 345 ms per loop


In [25]:
import numpy
@numba.vectorize
def my_function(a):
    return sin(a)*a + a**2 + 1/a

my_function(a)

array([ 2.35887454,  2.44498856,  3.10254861, ...,  2.36514506,
        5.13323677,  2.77822627])

In [26]:
%timeit my_function(a)

1 loop, best of 3: 200 ms per loop
